In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [59]:
def scrapeInsertTR(url, name):
    """Requests url, scrapes, finds text and rating, and inserts both into MongoDB
    
    **Input parameters**
    ------------------------------------------------------------------------------
    url: string.  Valid url to be requested.  This code is specifically built for 
            'http://www.everytrail.com'
    
    name: string. Name of the hike, scraped from parent function
    
    
    **Output**
    ------------------------------------------------------------------------------
    None
    
    """
    r = requests.get(url).text
    soup = BeautifulSoup(r, 'html.parser')
    if len(soup(attrs={'class': 'comment-content rounded floatright'})):
        for div in soup(attrs={'class': 'comment-content rounded floatright'}):
            rating = div(attrs={'itemprop':"reviewRating"})[0].span['title']
            text = div.p.text
    else:
        for div in soup(attrs={'style': 'overflow:hidden'}):
            if (div.span == None) or div.select('img') == []:
                continue
            else:
                rating = div.select('img')[0]['title'][0]
                text = div.span.text.strip()
    if len(text):
        db.trainingTR.insert_one({'Name': name,
                                   'Report': {
                                        "Rating": rating,
                                        "Text": text
                                        }
                                        })

In [60]:
def URLfinder(baseurl):
    """Finds all trailing URLs for hikes.
    
        Calls scrapeInsertTR for all trailing urls.
    
    **Input parameters**
    ------------------------------------------------------------------------------
    baseurl: string. Valid url to be requested.  This code is specifically built for 
            'http://www.everytrail.com/best/hiking-california'
    
    
    **Output**
    ------------------------------------------------------------------------------
    None
    
    """
    
    r = requests.get(baseurl).text
    soup = BeautifulSoup(r, 'html.parser')
    for div in soup.select('div.guide-preview-content'):
        url = 'http://www.everytrail.com'+div.span.a['href']
        scrapeInsertTR(url, div.span.a.text)

In [ ]:
if __name__ == '__main__':


In [61]:

client = MongoClient()
db = client.wta
db.trainingTR.drop()
URLfinder('http://www.everytrail.com/best/hiking-california')

http://www.everytrail.com/guide/pine-ridge-trail-to-sykes-hot-springs
http://www.everytrail.com/guide/cataract-trail-loop-in-marin-county
http://www.everytrail.com/guide/skyline-to-the-sea-trail
http://www.everytrail.com/guide/steep-ravine-via-matt-davis-mt-tamalpais-
http://www.everytrail.com/guide/berry-creek-falls-loop-at-big-basin-redwoods-state-park
http://www.everytrail.com/guide/the-mist-trail-vernal-and-nevada-falls
http://www.everytrail.com/guide/lake-aloha-hike
http://www.everytrail.com/guide/muir-woods-and-tourist-club-loop
http://www.everytrail.com/guide/the-panorama-trail
http://www.everytrail.com/guide/mount-tallac-lake-tahoe
http://www.everytrail.com/guide/mount-diablos-grand-loop
http://www.everytrail.com/guide/sturtevant-falls
http://www.everytrail.com/guide/batteries-to-bluffs-trail
http://www.everytrail.com/guide/stinson-beach-to-mt-tamalpais


UnboundLocalError: local variable 'text' referenced before assignment